In [8]:
# Load all the necessary modules

import os
import torch
import glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [9]:
# Load metadata CSV file
path = "/path/to/your/dataset/" 
mocapcsv_list = glob.glob(os.path.join(path, "*.csv"))


In [4]:
for item in mocapcsv_list:
    df = pd.read_csv(item)
    basename = os.path.splitext(os.path.basename(item))[0]

    LHAND_x = df['LHAND_x'].to_numpy()
    LHAND_y = df['LHAND_y'].to_numpy()
    LHAND_z = df['LHAND_z'].to_numpy()
    RHAND_x = df['RHAND_x'].to_numpy()
    RHAND_y = df['RHAND_y'].to_numpy()
    RHAND_z = df['RHAND_z'].to_numpy()

    # Compute velocity for each hand
    LHAND_velocity_x = np.gradient(LHAND_x)
    LHAND_velocity_y = np.gradient(LHAND_y)
    LHAND_velocity_z = np.gradient(LHAND_z)

    RHAND_velocity_x = np.gradient(RHAND_x)
    RHAND_velocity_y = np.gradient(RHAND_y)
    RHAND_velocity_z = np.gradient(RHAND_z)

    # Compute acceleration for each hand 
    LHAND_acceleration_x = np.gradient(LHAND_velocity_x)
    LHAND_acceleration_y = np.gradient(LHAND_velocity_y)
    LHAND_acceleration_z = np.gradient(LHAND_velocity_z)

    RHAND_acceleration_x = np.gradient(RHAND_velocity_x)
    RHAND_acceleration_y = np.gradient(RHAND_velocity_y)
    RHAND_acceleration_z = np.gradient(RHAND_velocity_z)

    # Stack the velocity and acceleration arrays for each hand
    mocap_feature = np.vstack((LHAND_velocity_x, LHAND_velocity_y, LHAND_velocity_z,
                               LHAND_acceleration_x, LHAND_acceleration_y, LHAND_acceleration_z,
                               RHAND_velocity_x, RHAND_velocity_y, RHAND_velocity_z,
                               RHAND_acceleration_x, RHAND_acceleration_y, RHAND_acceleration_z))


    np.save(f"/path/to/your/mocap/{basename}.npy", mocap_feature)

        

In [5]:
db_gesture = "/path/to/your/mocap/*"
files = glob.glob(db_gesture)
filename = [os.path.basename(item) for item in files]

target = [item.split("_")[-3] for item in files]

label_encoder = LabelEncoder()
target_idx = label_encoder.fit_transform(target) # Change class names to numeric

dict = {'mocap_npy': filename, 'target': target, 'target_idx': target_idx}
dataset_all = pd.DataFrame(dict)

dataset_all.to_csv('metadata.csv')

In [6]:
X = dataset_all['mocap_npy']
y = dataset_all['target_idx']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)

In [7]:
train_df = pd.DataFrame({'filename': list(X_train), 'target': list(y_train)})
test_df = pd.DataFrame({'filename':list(X_test), 'target': list(y_test)})

train_df.to_csv("/path/to/your/mocap/train.csv") 
test_df.to_csv("/path/to/your/mocap/test.csv") 